In [2]:
from __future__ import print_function
import numpy as np
import mxnet as mx
from mxnet import nd, autograd, gluon
from time import time
mx.random.seed(1)

In [3]:
ctx = mx.gpu()

In [4]:
batch_size = 64
num_inputs = 784
num_outputs = 10
def transform(data, label):
    return nd.transpose(data.astype(np.float32), (2,0,1))/255, label.astype(np.float32)

train_data = gluon.data.DataLoader(gluon.data.vision.MNIST(train=True, transform=transform),
                                      batch_size, shuffle=True)
test_data = gluon.data.DataLoader(gluon.data.vision.MNIST(train=False, transform=transform),
                                     batch_size, shuffle=False)


/usr/local/lib/python3.5/dist-packages/mxnet/gluon/data/vision/datasets.py:118: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  label = np.fromstring(fin.read(), dtype=np.uint8).astype(np.int32)
/usr/local/lib/python3.5/dist-packages/mxnet/gluon/data/vision/datasets.py:122: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(fin.read(), dtype=np.uint8)


In [5]:
num_fc = 512
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Conv2D(channels=20, kernel_size=5, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
    net.add(gluon.nn.Conv2D(channels=50, kernel_size=5, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
    # The Flatten layer collapses all axis, except the first one, into one axis.
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(num_fc, activation="relu"))
    net.add(gluon.nn.Dense(num_outputs))

In [6]:
net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

In [7]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [8]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .1})

In [9]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

In [10]:
epochs = 20
smoothing_constant = .01

for e in range(epochs):
    tic = time()
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(data.shape[0])

        ##########################
        #  Keep a moving average of the losses
        ##########################
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0))
                       else (1 - smoothing_constant) * moving_loss + smoothing_constant * curr_loss)

    test_accuracy = 0 #evaluate_accuracy(test_data, net)
    train_accuracy = 0 #evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s, Time %.1f sec" % (e, moving_loss, train_accuracy, test_accuracy, time()-tic))

Epoch 0. Loss: 0.08043880347708963, Train_acc 0, Test_acc 0, Time 30.0 sec
Epoch 1. Loss: 0.04992680303605025, Train_acc 0, Test_acc 0, Time 29.6 sec
Epoch 2. Loss: 0.03572506622418558, Train_acc 0, Test_acc 0, Time 29.4 sec
Epoch 3. Loss: 0.0260143448728486, Train_acc 0, Test_acc 0, Time 29.1 sec
Epoch 4. Loss: 0.01971091373484408, Train_acc 0, Test_acc 0, Time 29.3 sec
Epoch 5. Loss: 0.01726156820600957, Train_acc 0, Test_acc 0, Time 28.7 sec
Epoch 6. Loss: 0.015638464296622898, Train_acc 0, Test_acc 0, Time 28.6 sec
Epoch 7. Loss: 0.009331904507182194, Train_acc 0, Test_acc 0, Time 28.8 sec
Epoch 8. Loss: 0.009689594137781553, Train_acc 0, Test_acc 0, Time 29.8 sec
Epoch 9. Loss: 0.009646067989718676, Train_acc 0, Test_acc 0, Time 29.6 sec
Epoch 10. Loss: 0.005858414605580581, Train_acc 0, Test_acc 0, Time 28.5 sec
Epoch 11. Loss: 0.0047775944177645925, Train_acc 0, Test_acc 0, Time 28.9 sec
Epoch 12. Loss: 0.003453537663992551, Train_acc 0, Test_acc 0, Time 29.1 sec
Epoch 13. Loss:

In [13]:
net.save_params("models/cnn_1gpu_mnist.par")

In [12]:
data.shape[0]

32